In [34]:
import codecs
import re
from functions import compass
import pandas as pd
import numpy as np
from yahooquery import Ticker
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import date, timedelta
import altair as alt

In [12]:
link = ("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks")
sp500_raw = pd.read_html(link, header=0)[0]
sp500 = [str(stock) for stock in sp500_raw["Symbol"]]
data = compass(sp500)

In [13]:
base = alt.Chart(data)
xscale = alt.Scale(domain=(0.5, 1.5))
yscale = alt.Scale(domain=(0.5, 1.5))

points = base.mark_circle(color="red").encode(
    alt.X("Levy:Q").scale(xscale),
    alt.Y("Momentum:Q").scale(yscale),
)

text = points.mark_text(align='left',baseline='middle',dx=10).encode(
    text="Stock",
)

x_line = alt.Chart().mark_rule(strokeDash=[10, 10]).encode(y=alt.datum(1))
y_line = alt.Chart().mark_rule(strokeDash=[10, 10]).encode(x=alt.datum(1))


(points+text+x_line+y_line).properties(width=650,height=500, title="Trending Stock Indicator").interactive().save("Trendcompass.html")

NameError: name 'alt' is not defined

In [32]:
data["Mean"] = np.mean([data["Momentum"], data["Levy"]], axis=0)
new_data = data.sort_values(by=["Mean"], axis=0, ascending=False)[:30]

main_chart = alt.Chart(new_data, title="List of Best 30 Stocks").mark_point(color="red").encode(
alt.X("Mean:Q")
    .title("Levy: blue, Mean: red, Momentum: green")
    .scale(zero=False)
    .axis(grid=False),
alt.Y("Stock:N")
    .title("")
    .sort("-x")
    .axis(grid=True),

)

chart_1= alt.Chart(new_data).mark_point(color="green").encode(
alt.X("Momentum:Q")
    .scale(zero=False)
    .axis(grid=False),
alt.Y("Stock:N")
    .sort()
    .axis(grid=True),

)

chart_2 = alt.Chart(new_data).mark_point(color="blue").encode(
alt.X("Levy:Q")
    .scale(zero=False)
    .axis(grid=False),
alt.Y("Stock:N")
    .sort()
    .axis(grid=True),

)
(main_chart+chart_1+chart_2).properties(height=alt.Step(20)).configure_view(stroke="transparent").save("Scoreboard.html")

NameError: name 'data' is not defined

In [35]:
scatter = codecs.open("Trendcompass.html", "r").read()
board = codecs.open("Scoreboard.html", "r").read()

In [36]:
start = "spec = "
end = "var embedOpt = "
scatter = "".join(scatter.split(start)[1].split(end)[0])
board = "".join(board.split(start)[1].split(end)[0])


In [44]:
template = codecs.open("_layouts/backup.html", "r").read()
default = template.replace("*scatterplot_python*",scatter)
default = template.replace("*leaderboard_python*", board)

In [47]:
with open("_layouts/default.html", "w") as file:
    file.write(default)